In [2]:
import multiprocessing
import time

# Função global que será usada pelos processos de CPU
def cpu_task():
    while True:
        _ = 3.1415 * 2.7182  # Operação simples para manter a CPU ocupada

def stress_cpu():
    print("Estressando a CPU...")
    
    processes = []
    for _ in range(multiprocessing.cpu_count()):
        p = multiprocessing.Process(target=cpu_task)
        p.start()
        processes.append(p)

    time.sleep(10)  # Mantém o teste por 10 segundos
    for p in processes:
        p.terminate()
    print("Teste de CPU finalizado!")

# Executar o teste de CPU
stress_cpu()

Estressando a CPU...
Teste de CPU finalizado!


In [ ]:
#Teste Memoria Ram
import numpy as np

def stress_ram():
    print("Estressando a memória RAM...")
    arrays = []
    try:
        while True:
            arrays.append(np.random.random((1000, 1000)))  # Aloca arrays de 1.000 x 1.000 até a memória acabar
    except MemoryError:
        print("Memória RAM esgotada!")
    print("Teste de RAM finalizado!")

# Executar o teste de RAM
stress_ram()


Estressando a memória RAM...


In [ ]:
#Tetes GPU
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule

def stress_gpu():
    print("Estressando a GPU...")
    
    mod = SourceModule("""
    __global__ void stress(float *a)
    {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        a[idx] = a[idx] * a[idx];
    }
    """)
    
    stress_kernel = mod.get_function("stress")
    
    a = np.random.randn(4000000).astype(np.float32)
    
    for _ in range(100):
        stress_kernel(drv.Out(a), block=(512,1,1), grid=(4000,1))
    
    print("Teste de GPU finalizado!")

# Executar o teste de GPU
try:
    stress_gpu()
except ImportError:
    print("PyCUDA não está instalada ou a GPU não é compatível.")


In [ ]:
#Testes Disco
import os

def stress_disk():
    print("Estressando o disco (HD)...")
    file_path = "teste_hd.tmp"
    try:
        with open(file_path, "wb") as f:
            for _ in range(1000):
                f.write(os.urandom(1024 * 1024))  # Grava 1MB de dados aleatórios 1000 vezes
        print("Teste de escrita finalizado!")

        with open(file_path, "rb") as f:
            while f.read(1024 * 1024):
                pass  # Leitura do arquivo criado
        print("Teste de leitura finalizado!")
    finally:
        if os.path.exists(file_path):
            os.remove(file_path)  # Remove o arquivo temporário
    print("Teste de disco finalizado!")

# Executar o teste de disco
stress_disk()
